# Lab de Modelación 

In [79]:
#Librería a utilizar
import numpy as np
import surprise 
import pandas as pd
from numpy import linalg as LA

### Paso 1) Crear un sistema de recomendación (user-user).
#### a) Top 5 mejores libros rankeados.

In [2]:
#Definiendo mejores libros
df=pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False,encoding='latin-1')
df=df.drop('User-ID',axis=1)

#Restricción sobre cantidad de evaluaciones
contador=df.groupby('ISBN').count()
Filtrocontador=contador['Book-Rating']>=200
contador=contador[Filtrocontador]

df_final=pd.merge(df,contador,on='ISBN',how='inner')
df_final=df_final.drop('Book-Rating_y',axis=1)
df_final=df_final.groupby('ISBN').mean().sort_values('Book-Rating_x',ascending=False).head(5)
df_final=df_final.reset_index()
MejoresLibros=df_final['ISBN']
print(MejoresLibros)

0    043935806X
1    0439136369
2    0345339681
3    0446310786
4    059035342X
Name: ISBN, dtype: object


#### b) Sistema de recomendación user-user con similaridad del coseno.

In [139]:
#Limpia los valores no nulos y vuelve a realizar un acomodamiento
def Dataframe_final(ID_Usuario,test_df,df):
    
    #Filtros asociados a entradas no null y al usuario de interés
    filtro_nuevo=test_df.loc[ID_Usuario].notnull()==True
    
    #Libros en los cuál el usuario ha interactuado.
    indices=list(test_df.loc[ID_Usuario][filtro_nuevo].index)
    
    return(test_df[indices].fillna(0))
    
#Retorna el dataframe con una nueva columna "Similaridad" que describe la similaridad del coseno.
def Similaridad(ID_Usuario,test_df,df):
    df_usuario=Dataframe_final(ID_Usuario,test_df,df)
    largo_filas=df_usuario.shape[0]
    largo_columna=df_usuario.shape[1]
    df_usuario['Similaridad']=0
    for i in range(0,largo_filas):
        Producto_interno=np.dot(df_usuario.iloc[i,0:largo_columna-1],df_usuario.loc[ID_Usuario][0:largo_columna-1])
        Producto_norma=df_usuario.loc[ID_Usuario]['Norma']*df_usuario.iloc[i]['Norma']
        if Producto_norma==0:
            continue
        else:            
            df_usuario.iloc[i,largo_columna]=(Producto_interno/Producto_norma)
    return(df_usuario)

#Retorna los primeros n usuarios mas similares.
def Usuarios_Similares(ID_Usuario,test_df,df,n):
    df_usuarios=Similaridad(ID_Usuario,test_df,df)
    df_usuarios=df_usuarios.sort_values(by='Similaridad',ascending=False)
    return(df_usuarios.head(n).index)

#Retorna todos los items con los cuales el usuario ha interactuado
def Items_of_interest(df,ID_user):
     #Generando la independencia de nombre de las columnas
    columnas=list(df.columns)
    User=columnas[0]
    Item=columnas[1]
    Rating=columnas[2]
    
    #Filtro usuario a interés
    filtro_aux=df[User]==ID_user
    return list(df[filtro_aux][Item].unique())

#Norma de una fila (usuario en particular) de la matriz sparse.
def Norma(df,ID_user):
    #Generando la independencia de nombre de las columnas
    columnas=list(df.columns)
    User=columnas[0]
    Item=columnas[1]
    Rating=columnas[2]
    
    #Filtrando las participaciones del usuario a interés
    filtro_aux=df[User]==ID_user
    norma=LA.norm(df[filtro_aux][Rating])
    return norma

#Dado un usuario en particular pivotea (matriz sparse) para los casos que éste ha interactuado
def Particular_pivot(df,ID_user):
    #Independencia de nombres
    columnas=list(df.columns)
    User=columnas[0]
    Item=columnas[1]
    Rating=columnas[2]
    
    #Filtro de participación
    Filtro_usuario=df[User]==ID_user
    if df[Filtro_usuario].shape[0]<1:
        return('ID no registrada, no es posible encontrar usuarios similares')
    
    items=Items_of_interest(df,ID_user)
    test_df=df[df[Item].isin(items)].pivot(index=User,columns=Item,values=Rating)
    normas=[]
    for indice in list(test_df.index):
        normas.append(Norma(df,indice))
    test_df.loc[:,'Norma']=normas
    return test_df
    
def TopN_Singular(df,n,ID_user):
    test_df=Particular_pivot(df,ID_user)
    if isinstance(test_df, str):
        return test_df
    Top=Usuarios_Similares(ID_user,test_df,df,n)
    return list(Top)

def TopN_Plural(df,n,Usuarios):
    Tops=[]
    for Usuario in Usuarios:
        Tops.append(TopN_Singular(df,n,Usuario))
    return Tops

#Ejemplo para un dataframe en particular de libros.
df=pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False,encoding='latin-1')
df.sort_values('User-ID')

#Ejemplo con ID-Usuario 8,123629,200273
TopN_Plural(df,6,[8,123629,200273])

[[8, 123629, 200273, 210926, 67544, 226745],
 [123629, 200273, 210926, 67544, 8, 219008],
 [200273, 123629, 67979, 18486, 175420, 23131]]

### Caso de Laboratorio:

---
Se realiza un caso artifical, donde manualmente se verifica que sea capaz de encontrar usuarios similares

Donde se ve a simple vista que el usuario 1 y 2 son bastante similares y 2 y 3 no lo son.

---

In [135]:
datos={'User-ID':[1,1,1,2,1,3,3,2,2,4,2,3,4,4,4],
       'ISBN':['a','b','c','a','d','a','b','b','c','a','d','c','b','c','d'],
       'Book-Rating':[10,9,6,9,2,2,3,9,5,9,1,0,5,5,5]}
       
df_artificial=pd.DataFrame(datos)
display(Particular_pivot(df_artificial,2))
print(list(TopN_Singular(df_artificial,6,2)))


ISBN,a,b,c,d,Norma
User-ID,,,,,
1,10.0,9.0,6.0,2.0,14.866069
2,9.0,9.0,5.0,1.0,13.711309
3,2.0,3.0,0.0,NaN,3.605551
4,9.0,5.0,5.0,5.0,12.489996


[2, 1, 4, 3]


### Paso 2) 3 librerías de Surprise 

Las 3 librerías escogidas son: Normal Predictor, KNNWithMeans y Co-clustering

### Paso 3) Acomodar para futuros datos.

In [ ]:
#data=pd.read_csv('data_cust.csv')


 ### Paso 4) Crear un sistema de recomendación Item-Item

### Errores encontrados en la ejecución (Ignorar)

In [ ]:
# Problema Solucionado

#conteo_participacion=df.groupby('User-ID')['Book-Rating'].count().reset_index()
#conteo_participacion.columns.values[1]='Participacion'
#Filtroparticipacion=conteo_participacion['Participacion']>29
#conteo_participacion=conteo_participacion[Filtroparticipacion]
#display(conteo_participacion)
#df_2=pd.merge(df,conteo_participacion,on='User-ID',how='inner')
#df_2=df_2.drop(['Participacion'],axis=1)

#

#Problema 1: Incapacidad para segmentar los conjuntos de datos de forma de maximizar la aparición de 
#un usuario en ambos conjuntos
    
#Problema 2: Unable to allocate array with shape (140330835,) and data type int64 (Exceso de memoria ? )

#df_prueba=pd.merge(df_train,df_test,on='User-ID',how='inner')
#df_prueba=df_prueba.sample(frac=0.0001,replace=False,random_state=1)
#Usuarios=df_prueba.head(10).reset_index()['User-ID']

#for user in Usuarios:
    #i=0
    #df_train1=df_train
    #filtrouser=df_train1['User-ID']==user
    #Libros_calificados=df_train1[filtrouser][['ISBN','Book-Rating']]
    #Libros_calificados=Libros_calificados.sort_values('Book-Rating',ascending=False).head(5)
    #Libros=Libros_calificados['ISBN']
    #Recomendacion=MejoresLibros
    #for libro in Libros:
        #filtro_libro=df_train1['ISBN']==libro
        #if df_train1[filtro_libro].shape[0]<5:
            #i=-1
            #break
        #else:
            #df_train1=df_train1[filtro_libro]
    #if i==0:
        #df_train1=df_train1[~filtrouser]
        #df_train1.sort_values
        
    #print('Se le recomienda al usuario {} los siguientes libros {}'.format(user,Recomendacion))
    
    #test_df['Norma']=0
#test_df.columns
#for column in list(test_df.columns):
   # test_df['Norma']+=test_df[column]**2
#test_df
#new_df['Norma']=np.nan
#for column in list(new_df.columns):
   #new_df['Norma']+=new_df[column]**2